In [3]:
#generate data do compare network quality
#safe average differences between wt and mut population
import sys
sys.path.insert(0, '../../helperScripts/')
from helperFunctions import *
import scmra
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import pandas as pd
from datetime import datetime
import pickle
import seaborn as sns; sns.set_theme()
from matplotlib.pyplot import figure
import scmra    


In [8]:
def get_population_list_from_string(string):
   popList = []
   x = string.split(",")
   for pop in x:
      if pop == "": 
         continue
      else:
         popList.append(pop)
   return(popList)

def calculate_diff_dict():
   dict = {}
   dict['braf'] = ((rloc_true - rloc_braf)**2)**0.5
   dict['ras'] = ((rloc_true - rloc_ras)**2)**0.5

   return(dict)

def extract_average_rloc_differences(pickleFile, outputFile,):
   result = pickle.load(open(pickleFile, 'rb'))

   diffDict = calculate_diff_dict()

   # Create the pandas DataFrame
   df = pd.DataFrame(columns=["POPULATIONS", "rloc", "value"] + result.header)
   
   # for every combination of
   # NOISE CELLNUM REPEAT of CNR
   headers = result.header
   resultFrame = pd.DataFrame(columns=headers)
   #add all the existing parameters
   resultNumber = len(result.parameter)
   for i in range(resultNumber):
      params = result.parameter[i]
      df_length = len(resultFrame)
      resultFrame.loc[df_length] = params
   cnrData = resultFrame[resultFrame["MODELTYPE"] == "CNR"]

   #get noise cellnum and repeats for CNR
   noiseLevels = cnrData["NOISE"].unique()
   cellnumLevels = cnrData["CELLNUM"].unique()
   repeatLevels = cnrData["REPEAT"].unique()
   thetaLevels = cnrData["THETA"].unique()

   #get populations that r compared for CNR (taking only the ones with 2 populations for now)
   populationLevelsTmp = cnrData["POPULATIONS"].unique()
   populationLevels = []
   for level in populationLevelsTmp:
      x = level.split(",")
      populationNumber = 0
      for pop in x:
         if pop == "": continue
         populationNumber += 1

      if(populationNumber == 2):
         populationLevels.append(level)

   #---------------------------
   #CNR
   #---------------------------
   keyList = ["MODELTYPE", "NOISE", "CELLNUM", "POPULATIONS","REPEAT","THETA","EDGES"]
   for noise in noiseLevels:
      for cellnum in cellnumLevels:
         for pop in populationLevels:
            for repeat in repeatLevels:
               for theta in thetaLevels:
                  valueList = ["CNR", noise, cellnum, pop, repeat, theta, 13]
                  popList = get_population_list_from_string(pop) # list of populations like ["wt", "braf"]
                  if(len(popList) != 2): 
                     print("NOT EXACTLY 2 POPULATIONS FOR MRA -> SKIP MRA RLOC DIFF CALCUALTION\n")
                     continue
                  if(not "wt" in popList and not "WT" in popList): 
                     print("NO wildtype <wt> in population list (needed for rloc Diff) -> SKIP MRA RLOC DIFF CALCUALTION \n")
                     continue

                  resultObject, params = result.getResult(keyList, valueList) #the result from the simulation
                  if(resultObject == None): continue

                  vars_lst = [var for var in resultObject.vardict if var.startswith('IDev')]
                  vars_vals = [resultObject.vardict[v] for v in vars_lst]
                  lengthIdcs = (np.count_nonzero(vars_vals))

                  assert(len(popList) == 2)
                  rloc_list = {}
                  #we assign population by wt, braf, ras. Those MUST BE LOWER CASE NAMES.
                  #just in case convert to lower case beforehand
                  lowerPopList = []
                  for popTmp in popList:
                     lowerPopList.append(popTmp.lower())
                  popList = lowerPopList
                  for popStringIdx in range(len(popList)):
                     popString = popList[popStringIdx]
                     rloc_tmp = resultObject.rloc[popString]
                     rloc_tmp = reorder_rloc(rloc_tmp)
                     rloc_list[popList[popStringIdx]] = (rloc_tmp)

                  # get all rloc differences and write one line per difference
                  columns = (rloc_true.columns)
                  rows = (rloc_true.index)

                  if("braf" in popList):
                     reconstuctedDiff = rloc_list["braf"] - rloc_list["wt"]
                  elif("ras" in popList):
                     reconstuctedDiff = rloc_list["ras"] - rloc_list["wt"]
                  
                  for col in columns:
                     for row in rows:
                        #difference is defined as mutant - wildtype
                        value = reconstuctedDiff.at[row, col]
                        rloc = row + "_" + col

                        #write new row in result data frame
                        # dataframe struture is: columns=["NOISE", "CELLNUM", "REPEAT", "POPULATIONS", "DIFFERENCE_RMSE"]
                        list1 = [str(popList[0]) + "_" + str(popList[1]), rloc, value]
                        list2 = params
                        rowList = [*list1, *list2] 
                        df.loc[len(df)] = rowList

   #---------------------------
   #MRA
   #---------------------------
   headers = result.header
   resultFrame = pd.DataFrame(columns=headers)
   #add all the existing parameters
   resultNumber = len(result.parameter)
   for i in range(resultNumber):
      params = result.parameter[i]
      df_length = len(resultFrame)
      resultFrame.loc[df_length] = params
   mraData = resultFrame[resultFrame["MODELTYPE"] == "MRA"]

   if(not mraData.empty):

      #get all possible populations to look into for MRA
      keyList = ["MODELTYPE", "NOISE", "CELLNUM", "POPULATIONS","REPEAT"]

      for popList in [ ["WT:wt_braf", "BRAF:wt_braf"], ["WT:wt_ras","RAS:wt_ras"]]:
         for repeatId in repeatLevels:
            for noise in noiseLevels:
               for cellnum in cellnumLevels:

                  #GET FIRST RESULT
                  valueList = ["MRA", noise, cellnum, popList[0], repeatId]
                  result1, params1 = result.getResult(keyList, valueList, False, True) #the result from the simulation

                  assert(len(result1) == 1)           
                  result1 = result1[0]
                  params1 = params1[0]
                  popListSingle = popList[0].split(":")
                  pop1 = popListSingle[0]
                  assert(pop1 == "WT")

                  #GET SECOND RESULT
                  valueList = ["MRA", noise, cellnum, popList[1], repeatId]
                  result2, params2 = result.getResult(keyList, valueList, False, True) #the result from the simulation
                  assert(len(result2) == 1)           
                  result2 = result2[0]
                  params2 = params2[0]
                  popListSingle = popList[1].split(":")
                  pop2 = popListSingle[0]
                  assert(pop2 == "BRAF" or pop2 == "RAS")

                  print(pop1)
                  print(pop2)

                  populationList = []

                  populationList.append(pop1.lower())
                  populationList.append(pop2.lower())
                  #calculate RMSE for ALL COMBINATIONS

                  rloc_list = {}
                  columns = (rloc_true.columns)
                  rows = (rloc_true.index)
                  rloc_tmp1 = reorder_rloc(result1.rloc)
                  rloc_tmp2 = reorder_rloc(result2.rloc)
                  rloc_list["wt"] = (rloc_tmp1)
                  if(pop2 == "BRAF"):
                     rloc_list["braf"] = (rloc_tmp2)
                     reconstuctedDiff = rloc_list["braf"] - rloc_list["wt"]
                  elif(pop2 == "RAS"):
                     rloc_list["ras"] = (rloc_tmp2)
                     reconstuctedDiff = rloc_list["ras"] - rloc_list["wt"]
                  
                  for col in columns:
                     for row in rows:
                        #difference is defined as mutant - wildtype
                        value = reconstuctedDiff.at[row, col]
                        rloc = row + "_" + col

                        #write new row in result data frame
                        # dataframe struture is: columns=["NOISE", "CELLNUM", "REPEAT", "POPULATIONS", "DIFFERENCE_RMSE"]
                        list1 = [str(pop1) + "_" + str(pop2), rloc, value]
                        list2 = params1
                        rowList = [*list1, *list2] 
                        df.loc[len(df)] = rowList

   #finally print the dataframe
   df.to_csv(outputFile, index=False, sep="\t")

In [9]:
pickleFile = "/DATA/t.stohn/MRA/scMRA-analysis/notebooks/paperAnalyses/2_CNRAnalysis/bin/SIMULATION_CnrOverMra_NoiseRange_CellNum250_04-19-2023|20:59:39.pickle"
result = pickle.load(open(pickleFile, 'rb'))

extract_average_rloc_differences(pickleFile, "./bin/AverageRlocDifference.txt")

WT
BRAF
['WT_BRAF', 'Sos_Sos', 0.0, 0.0, 250, 0.00625, 0, 13, 'MRA', 'WT:wt_braf', 0, 'NO_THETA', 6.14997963409419e-05]
['WT_BRAF', 'Ras_Sos', -0.012468785029130514, 0.0, 250, 0.00625, 0, 13, 'MRA', 'WT:wt_braf', 0, 'NO_THETA', 6.14997963409419e-05]
['WT_BRAF', 'Raf1_Sos', 0.0, 0.0, 250, 0.00625, 0, 13, 'MRA', 'WT:wt_braf', 0, 'NO_THETA', 6.14997963409419e-05]
['WT_BRAF', 'Mek_Sos', 0.0, 0.0, 250, 0.00625, 0, 13, 'MRA', 'WT:wt_braf', 0, 'NO_THETA', 6.14997963409419e-05]
['WT_BRAF', 'Erk_Sos', 0.0, 0.0, 250, 0.00625, 0, 13, 'MRA', 'WT:wt_braf', 0, 'NO_THETA', 6.14997963409419e-05]
['WT_BRAF', 'P90Rsk_Sos', 0.0, 0.0, 250, 0.00625, 0, 13, 'MRA', 'WT:wt_braf', 0, 'NO_THETA', 6.14997963409419e-05]
['WT_BRAF', 'PI3K_Sos', 0.0, 0.0, 250, 0.00625, 0, 13, 'MRA', 'WT:wt_braf', 0, 'NO_THETA', 6.14997963409419e-05]
['WT_BRAF', 'Akt_Sos', 0.0, 0.0, 250, 0.00625, 0, 13, 'MRA', 'WT:wt_braf', 0, 'NO_THETA', 6.14997963409419e-05]
['WT_BRAF', 'C3G_Sos', 0.0, 0.0, 250, 0.00625, 0, 13, 'MRA', 'WT:wt_braf'